In [1]:
import os
import pandas as pd

from ftplib import FTP

In [2]:
data_folder = f"../data"

sample_filename = "muestreos_parcelas_utm.parquet"

df_samples = pd.read_parquet(f"{data_folder}/{sample_filename}")
#df_samples.head(20).to_csv("sample_parcelas.txt", sep="\t")

df_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581793 entries, 0 to 581792
Data columns (total 62 columns):
 #   Column                                              Non-Null Count   Dtype         
---  ------                                              --------------   -----         
 0   generated_muestreos                                 581793 non-null  datetime64[us]
 1   codparcela                                          581793 non-null  category      
 2   provincia                                           581793 non-null  string        
 3   municipio                                           581793 non-null  object        
 4   fecha                                               581793 non-null  datetime64[us]
 5   campaña                                             581793 non-null  uint16        
 6   poligono                                            581731 non-null  UInt16        
 7   parcela                                             581763 non-null  UInt16        

In [3]:
geographic_colums = [
       'generated_muestreos',
       'fecha',
       'codparcela',
       '102_coordenada_x_(utm)',
       '103_coordenada_y_(utm)',
       '104_altitud_(m)',
       '201_superf_cultivada_en_la_parcela_agrícola_(ha)',
       '203_orientación',
       "huso_etrs89_regcan95"
       ]

df_space_samples = df_samples[geographic_colums].copy()

df_space_samples.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581793 entries, 0 to 581792
Data columns (total 9 columns):
 #   Column                                            Non-Null Count   Dtype         
---  ------                                            --------------   -----         
 0   generated_muestreos                               581793 non-null  datetime64[us]
 1   fecha                                             581793 non-null  datetime64[us]
 2   codparcela                                        581793 non-null  category      
 3   102_coordenada_x_(utm)                            570857 non-null  float64       
 4   103_coordenada_y_(utm)                            570759 non-null  float64       
 5   104_altitud_(m)                                   389024 non-null  float32       
 6   201_superf_cultivada_en_la_parcela_agrícola_(ha)  530584 non-null  float32       
 7   203_orientación                                   244094 non-null  category      
 8   huso_etrs89_reg

In [4]:
df_space_samples.head()

,generated_muestreos,fecha,codparcela,102_coordenada_x_(utm),103_coordenada_y_(utm),104_altitud_(m),201_superf_cultivada_en_la_parcela_agrícola_(ha),203_orientación,huso_etrs89_regcan95
0,2020-04-23 14:30:39,2006-04-16,023-00109-00-00,520481.06,4111107.12,863.0,2.0,NaN,30
1,2020-04-23 14:30:39,2006-04-27,023-00109-00-00,520481.06,4111107.12,863.0,2.0,NaN,30
2,2020-04-23 14:30:39,2006-05-04,023-00109-00-00,520481.06,4111107.12,863.0,2.0,NaN,30
3,2020-04-23 14:30:39,2006-05-11,023-00109-00-00,520481.06,4111107.12,863.0,2.0,NaN,30
4,2020-04-23 14:30:39,2006-05-17,023-00109-00-00,520481.06,4111107.12,863.0,2.0,NaN,30


In [6]:
group_dates = df_space_samples.groupby("codparcela")["fecha"].agg([('min_fecha', 'min'), ('max_fecha', 'max')]).reset_index()
group_dates.head()

/tmp/ipykernel_179235/2784411242.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_dates = df_space_samples.groupby("codparcela")["fecha"].agg([('min_fecha', 'min'), ('max_fecha', 'max')]).reset_index()


,codparcela,min_fecha,max_fecha
0,000-00000-00,2015-08-25,2015-09-30
1,000-00000-00-0,2015-08-24,2015-09-29
2,000-00900-00-U0,2006-04-24,2009-11-16
3,000-00999-00-00,2006-05-02,2009-11-18
4,000-00999-00-FC,2006-05-02,2006-10-31


In [10]:
df_space_samples['año'] = df_space_samples['fecha'].dt.year
group_dates = df_space_samples.groupby(["codparcela", "año"])["fecha"].agg([('min_fecha', 'min'), ('max_fecha', 'max')]).reset_index()
group_dates.to_csv("parcelas_anho.txt", sep="\t")

/tmp/ipykernel_179235/279430687.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_dates = df_space_samples.groupby(["codparcela", "año"])["fecha"].agg([('min_fecha', 'min'), ('max_fecha', 'max')]).reset_index()


In [5]:
group_coords =df_space_samples.groupby("codparcela")[["102_coordenada_x_(utm)", "103_coordenada_y_(utm)"]].apply("first").reset_index()
group_coords.head()

/tmp/ipykernel_172047/2087360051.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_coords =df_space_samples.groupby("codparcela")[["102_coordenada_x_(utm)", "103_coordenada_y_(utm)"]].apply("first").reset_index()


,codparcela,102_coordenada_x_(utm),103_coordenada_y_(utm)
0,000-00000-00,276171.00,4082574.00
1,000-00000-00-0,232382.86,4072834.94
2,000-00900-00-U0,449521.00,4124050.00
3,000-00999-00-00,449772.00,4123208.00
4,000-00999-00-FC,449772.00,4123208.00


In [6]:
result = pd.merge(group_dates, group_coords, on='codparcela', how='inner')

result["days"] = (result["max_fecha"] - result["min_fecha"]).dt.days

result.to_csv("parcelas.txt", "\t")

In [7]:
municipios_list = df_samples["municipio"].unique()

len(municipios_list)

326

In [8]:
df_utm = pd.read_csv(f"../data/ngbe.csv", sep=";")
df_utm.head()



/tmp/ipykernel_172047/3120916277.py:1: DtypeWarning: Columns (1,3,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df_utm = pd.read_csv(f"../data/ngbe.csv", sep=";")


,id,codigo_ngbe,codigo_ngbe_text,nombre_extendido,identificador_geografico,nombre_alternativo_2,nombre_alternativo_3,nombre_variante_1,nombre_variante_2,nombre_variante_3,...,estatus_variante_3,long_etrs89_regcan95,lat_etrs89_regcan95,huso_etrs89_regcan95,x_utm_etrs89_regcan95,y_utm_etrs89_regcan95,hojamtn_25,codigo_ine,provincias_nombre,provincias_id
0,3000000,1.0,División administrativa.Estado,NaN,España,NaN,NaN,NaN,NaN,NaN,...,NaN,-3.685424,40.307923,30.0,441755.83,4462159.91,0582c2,00000,Estatal,00
1,2720806,1.1.1,División administrativa.Comunidad autónoma,Cataluña/Catalunya,Cataluña,Catalunya,NaN,NaN,NaN,NaN,...,NaN,2.166687,41.383484,31.0,430324.55,4581663.81,0421c3,"08000,17000,25000,43000","Tarragona,Barcelona,Lleida,Girona","43,08,25,17"
2,2720814,1.1.1,División administrativa.Comunidad autónoma,País Vasco/Euskadi,País Vasco,Euskadi,NaN,NaN,NaN,NaN,...,NaN,-2.666687,42.850098,30.0,527233.75,4744222.52,0112c4,"01000, 20000, 48000","Bizkaia,Araba/Álava,Gipuzkoa","48,01,20"
3,2720800,1.1.1,División administrativa.Comunidad autónoma,NaN,Andalucía,NaN,NaN,NaN,NaN,NaN,...,NaN,-5.983276,37.383484,30.0,235867.22,4141591.98,0984c4,"04000,11000,14000,18000,21000,23000,29000,41000","Málaga,Granada,Almería,Cádiz,Jaén,Córdoba,Huel...","29,18,04,11,23,14,21,41"
4,2720804,1.1.1,División administrativa.Comunidad autónoma,NaN,Castilla y León,NaN,NaN,NaN,NaN,NaN,...,NaN,-4.716674,41.650086,30.0,357050.36,4612350.23,0372c1,"05000,09000,24000,34000,37000,40000,42000,4700...","Segovia,Ávila,Palencia,Valladolid,Burgos,León,...","40,05,34,47,09,24,49,42,37"


In [12]:
df_utm = df_utm[df_utm["codigo_ngbe_text"] == 'División administrativa.Municipio']

df_utm["identificador_geografico"] = df_utm["identificador_geografico"].str.lower()

df_municipios = df_utm[df_utm["identificador_geografico"].isin(municipios_list)]
df_municipios.rename(columns={"identificador_geografico":"municipio"}, inplace=True)

df_municipios = df_municipios[["municipio", "codigo_ngbe_text", "huso_etrs89_regcan95", 'long_etrs89_regcan95', 'lat_etrs89_regcan95','x_utm_etrs89_regcan95','y_utm_etrs89_regcan95']]

df_municipios.drop_duplicates()

/tmp/ipykernel_172047/3537484760.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_municipios.rename(columns={"identificador_geografico":"municipio"}, inplace=True)


,municipio,codigo_ngbe_text,huso_etrs89_regcan95,long_etrs89_regcan95,lat_etrs89_regcan95,x_utm_etrs89_regcan95,y_utm_etrs89_regcan95
108,alameda,División administrativa.Municipio,30.0,-4.660269,37.209459,352676.380947,4.119400e+06
146,alfarnatejo,División administrativa.Municipio,30.0,-4.272012,36.979574,386788.543533,4.093362e+06
232,villanueva del arzobispo,División administrativa.Municipio,30.0,-3.011866,38.169629,498960.592428,4.224636e+06
522,baena,División administrativa.Municipio,30.0,-4.328959,37.613188,382708.067290,4.163729e+06
863,oria,División administrativa.Municipio,30.0,-2.296021,37.483602,562238.743901,4.148755e+06
...,...,...,...,...,...,...,...
7856,larva,División administrativa.Municipio,30.0,-3.201424,37.759013,482257.830330,4.179097e+06
7859,almargen,División administrativa.Municipio,30.0,-5.021139,37.002114,320161.585093,4.097016e+06
7950,purullena,División administrativa.Municipio,30.0,-3.189240,37.317719,483232.464909,4.130136e+06
8116,huesa,División administrativa.Municipio,30.0,-3.075197,37.765566,493376.964159,4.179807e+06


In [13]:
df_municipios.head()

,municipio,codigo_ngbe_text,huso_etrs89_regcan95,long_etrs89_regcan95,lat_etrs89_regcan95,x_utm_etrs89_regcan95,y_utm_etrs89_regcan95
108,alameda,División administrativa.Municipio,30.0,-4.660269,37.209459,352676.380947,4.119400e+06
146,alfarnatejo,División administrativa.Municipio,30.0,-4.272012,36.979574,386788.543533,4.093362e+06
232,villanueva del arzobispo,División administrativa.Municipio,30.0,-3.011866,38.169629,498960.592428,4.224636e+06
522,baena,División administrativa.Municipio,30.0,-4.328959,37.613188,382708.067290,4.163729e+06
863,oria,División administrativa.Municipio,30.0,-2.296021,37.483602,562238.743901,4.148755e+06


In [14]:
df_samples_reduced = df_samples[["municipio", "provincia"]]
df_samples_reduced.drop_duplicates(inplace=True)

/tmp/ipykernel_172047/746937361.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_samples_reduced.drop_duplicates(inplace=True)


In [15]:
df_husos = pd.merge(df_samples_reduced, df_municipios, on='municipio', how='left')

In [17]:
df_husos.head()

df_husos.to_csv("huso_municipios.txt", sep="\t")

In [11]:
df_husos = pd.read_csv("huso_municipios.txt", sep="\t", index_col=0)

df_husos.head()



,municipio,provincia,codigo_ngbe_text,huso_etrs89_regcan95,long_etrs89_regcan95,lat_etrs89_regcan95,x_utm_etrs89_regcan95,y_utm_etrs89_regcan95
0,abla,almeria,División administrativa.Municipio,30,-2.779540,37.141210,519579.3849,4110560.174
1,abrucena,almeria,División administrativa.Municipio,30,-2.797467,37.132183,517989.3667,4109555.201
2,antas,almeria,División administrativa.Municipio,30,-1.917153,37.244592,596039.5078,4122555.585
3,armuna de almanzora,almeria,NaN,30,NaN,NaN,NaN,NaN
4,finana,almeria,NaN,30,NaN,NaN,NaN,NaN


In [23]:
df_samples_completed = pd.merge(df_samples, df_husos, on="municipio", how="left")

In [26]:
df_samples_completed.to_parquet("../data/muestreos_parcelas_utm.parquet")

In [12]:
df_samples = pd.read_parquet("../data/muestreos_parcelas_utm.parquet")
df_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581793 entries, 0 to 581792
Data columns (total 62 columns):
 #   Column                                              Non-Null Count   Dtype         
---  ------                                              --------------   -----         
 0   generated_muestreos                                 581793 non-null  datetime64[us]
 1   codparcela                                          581793 non-null  category      
 2   provincia                                           581793 non-null  string        
 3   municipio                                           581793 non-null  object        
 4   fecha                                               581793 non-null  datetime64[us]
 5   campaña                                             581793 non-null  uint16        
 6   poligono                                            581731 non-null  UInt16        
 7   parcela                                             581763 non-null  UInt16        

In [15]:
df_muestra = df_samples[["codparcela", "provincia", "municipio", "102_coordenada_x_(utm)", "103_coordenada_y_(utm)", "huso_etrs89_regcan95"]]

In [43]:
df_muestra[(df_muestra["huso_etrs89_regcan95"]==29) & (df_muestra["102_coordenada_x_(utm)"]>999999)].dropna()

,codparcela,provincia,municipio,102_coordenada_x_(utm),103_coordenada_y_(utm),huso_etrs89_regcan95
540187,017-00002-02,sevilla,bollullos de la mitacion,74880420.0,413185045.0,29
540188,017-00002-02,sevilla,bollullos de la mitacion,74880420.0,413185045.0,29
540189,017-00002-02,sevilla,bollullos de la mitacion,74880420.0,413185045.0,29
540190,017-00002-02,sevilla,bollullos de la mitacion,74880420.0,413185045.0,29
540191,017-00002-02,sevilla,bollullos de la mitacion,74880420.0,413185045.0,29
540192,017-00002-02,sevilla,bollullos de la mitacion,74880420.0,413185045.0,29
540193,017-00002-02,sevilla,bollullos de la mitacion,74880420.0,413185045.0,29
540194,017-00002-02,sevilla,bollullos de la mitacion,74880420.0,413185045.0,29
547268,010-00026-01,sevilla,espartinas,75426391.0,414212601.0,29
547269,010-00026-01,sevilla,espartinas,75426391.0,414212601.0,29


In [47]:
import utm
utm.to_latlon(74880.4, 41318.5, 30, 'S')

OutOfRangeError: easting out of range (must be between 100,000 m and 999,999 m)

In [38]:
1e9

1000000000.0

In [ ]:
from pyproj import Proj, transform
Proj()